In [10]:
#from taucmdr.cf.storage.levels import USER_STORAGE
#from taucmdr.error import ConfigurationError
#from taucmdr.model.trial import Trial
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import nbformat
import plotly.graph_objs as pgo
from tau_profile_parser import TauProfileParser
import matplotlib
import pandas as pd

In [ ]:
# TRIAL BAR PLOT

def trial_barplot_figure(trial_id, metric):
    profile = TauProfileParser.parse('./mpi-profile-example')

    mdata = profile.interval_data()[[metric]]
    idata = mdata.unstack().sort_values((0, 0, 0), axis = 1, ascending = False).transpose().fillna(0)
    df = pd.concat([idata[:100], idata[:100].sum(level = 0).rename({metric: 'MISC'})], copy = False)
    indices = [str(i) for i in df.columns.values]

    data = [
        pgo.Bar(
            y = indices[::-1],
            x = r.values[::-1],
            name = idx[1],
            orientation = "h",
            hoverinfo = "text",
            text = [ "<br>".join([idx[1][i:i+75] for i in range(0, len(idx[1]), 75)])] * len(indices)
        ) for idx, r in df.iterrows()]

    data[-1]['marker'] = { 'color': '#333333' }

    layout = pgo.Layout(
        barmode = 'stack',
        showlegend = False,
        hovermode = 'closest',
        margin = pgo.layout.Margin(l = 0, r = 0, b = 0, t = 10))

    return pgo.Figure(data = data, layout = layout)

def trial_barplot_widget(trial_id, metric):
    return trial_barplot_figure(trial_id, metric)

trial_selector = widgets.Dropdown(options=[1,2,3], value=1)
metric_selector = widgets.Dropdown(options=["Exclusive", "Inclusive"], value="Exclusive")
interact(trial_barplot_widget, trial_id = trial_selector, metric = metric_selector)

In [ ]:
# PROFILE BAR PLOT

def profile_barplot_figure(trial_id, metric, indices):
    profile = TauProfileParser.parse('./mpi-profile-example')
    idata = profile.interval_data()
    
    ds = idata.loc[indices][[metric]].sort_values(metric)
    maxval = ds[metric].max()
    nrows = ds.shape[0]
    
    colors_str = "#1f77b4,#aec7e8,#ff7f0e,#ffbb78,#2ca02c,#98df8a,#d62728,#ff9896,#9467bd,#c5b0d5,#8c564b,#c49c94,#e377c2,#f7b6d2,#7f7f7f,#c7c7c7,#bcbd22,#dbdb8d,#17becf,#9edae5,#7fc97f,#beaed4,#fdc086,#ffff99,#386cb0,#f0027f,#bf5b17,#666666,#1b9e77,#d95f02,#7570b3,#e7298a,#66a61e,#e6ab02,#a6761d,#666666,#fbb4ae,#b3cde3,#ccebc5,#decbe4,#fed9a6,#ffffcc,#e5d8bd,#fddaec,#f2f2f2,#393b79,#5254a3,#6b6ecf,#9c9ede,#637939,#8ca252,#b5cf6b,#cedb9c,#8c6d31,#bd9e39,#e7ba52,#e7cb94,#843c39,#ad494a,#d6616b,#e7969c,#7b4173,#a55194,#ce6dbd,#de9ed6,#8dd3c7,#ffffb3,#bebada,#fb8072,#80b1d3,#fdb462,#b3de69,#fccde5,#d9d9d9,#bc80bd,#ccebc5,#ffed6f,#3182bd,#6baed6,#9ecae1,#c6dbef,#e6550d,#fd8d3c,#fdae6b,#fdd0a2,#31a354,#74c476,#a1d99b,#c7e9c0,#756bb1,#9e9ac8,#bcbddc,#dadaeb,#636363,#969696,#bdbdbd,#d9d9d9"
    colors = colors_str.split(',')

    data = [
        pgo.Bar(
            x = ds[metric],
            y = ds.index.values,
            orientation = 'h',
            marker = dict(color = colors[:nrows]),
            text = ds.index.values,
            textposition = 'outside'
        )]

    layout = pgo.Layout(
        autosize = True,
        height = 25 * nrows,
        margin = pgo.layout.Margin(l = 0, r = 0, b = 0, t = 10),
        xaxis = dict(range = (maxval * -0.1, maxval * 1.25)))

    return pgo.Figure(data = data, layout = layout)

def profile_barplot_widget(trial_id, metric, indices):
    return profile_barplot_figure(trial_id, metric, indices)

trial_selector = widgets.Dropdown(options = [1, 2, 3], value = 1)
metric_selector = widgets.Dropdown(options = ["Exclusive", "Inclusive"], value = "Exclusive")
indices_selector = widgets.Dropdown(options = [(0, 0, 0), (1, 0, 0)], value = (0, 0, 0))
interact(profile_barplot_widget, trial_id = trial_selector, metric = metric_selector, indices = indices_selector)

In [ ]:
# RUNTIME BREAKDOWN

def runtime_breakdownplot_figure(trials, metric, topn):
    dfs = []
    for id in trials:
        df = TauProfileParser.parse('.tau/mpi-examples/ring-sample/{}'.format(id))
        df = df.interval_data().reset_index()
        df['Ranks'] = [df["Node"].nunique()] * df["Node"].shape[0]
        df["Timer"] = df["Timer"].apply(lambda x: x.strip())
        timer_total = df[metric].sum()
        df["Normed_Metric"] = df[metric].apply(lambda x: round(float(x) / timer_total, 6))
        df = df.groupby(["Ranks", "Timer"])["Normed_Metric"].sum().drop(columns=["index"]).reset_index()
        dfs.append(df)

    mdfs = pd.concat(dfs)

    rank_vals = mdfs["Ranks"].unique()
    rank_max = mdfs["Ranks"].max()
    topn_timers = mdfs[mdfs["Ranks"] == rank_max].sort_values("Normed_Metric", ascending=False)[:topn]["Timer"]

    mdfs = mdfs[mdfs["Timer"].isin(topn_timers)]

    for rank in rank_vals:
        normsum = mdfs[mdfs["Ranks"] == rank]["Normed_Metric"].sum()
        remsum = 1 - normsum
        mdfs = mdfs.append({"Ranks": rank, "Timer": "Misc", "Normed_Metric": remsum}, ignore_index=True)

    mdfs = mdfs.pivot(index='Timer', columns='Ranks', values='Normed_Metric')
    mdfs = mdfs.sort_values(rank_max, ascending=False)

    ranks = list(mdfs.columns)
    times = mdfs.values.tolist()
    timers = mdfs.index.values

    colors_str = "#1f77b4,#aec7e8,#ff7f0e,#ffbb78,#2ca02c,#98df8a,#d62728,#ff9896,#9467bd,#c5b0d5,#8c564b,#c49c94,#e377c2,#f7b6d2,#7f7f7f,#c7c7c7,#bcbd22,#dbdb8d,#17becf,#9edae5,#7fc97f,#beaed4,#fdc086,#ffff99,#386cb0,#f0027f,#bf5b17,#666666,#1b9e77,#d95f02,#7570b3,#e7298a,#66a61e,#e6ab02,#a6761d,#666666,#fbb4ae,#b3cde3,#ccebc5,#decbe4,#fed9a6,#ffffcc,#e5d8bd,#fddaec,#f2f2f2,#393b79,#5254a3,#6b6ecf,#9c9ede,#637939,#8ca252,#b5cf6b,#cedb9c,#8c6d31,#bd9e39,#e7ba52,#e7cb94,#843c39,#ad494a,#d6616b,#e7969c,#7b4173,#a55194,#ce6dbd,#de9ed6,#8dd3c7,#ffffb3,#bebada,#fb8072,#80b1d3,#fdb462,#b3de69,#fccde5,#d9d9d9,#bc80bd,#ccebc5,#ffed6f,#3182bd,#6baed6,#9ecae1,#c6dbef,#e6550d,#fd8d3c,#fdae6b,#fdd0a2,#31a354,#74c476,#a1d99b,#c7e9c0,#756bb1,#9e9ac8,#bcbddc,#dadaeb,#636363,#969696,#bdbdbd,#d9d9d9"
    colors = colors_str.split(',')

    data = []
    for i, time_vals in enumerate(times):
        data.append(pgo.Scatter(
            x = rank_vals,
            y = time_vals,
            mode = 'lines',
            line = dict(width = 0.5, color = colors[i]),
            stackgroup = 'one',
            name = timers[i][:75],
            text = timers[i][:75],
            hoverinfo = 'text',
            groupnorm = 'percent'
        ))

    layout = pgo.Layout(
        hovermode = 'closest',
        showlegend = False,
        xaxis_type = 'category',
        yaxis = dict(
            type = 'linear',
            range = [1, 100],
            ticksuffix='%'))

    fig = pgo.Figure(data = data, layout = layout)
    return fig

def runtime_breakdownplot_widget(trials, metric, topn):
    return runtime_breakdownplot_figure(trials, metric, topn)

trial_selector = widgets.SelectMultiple(options = [0, 1, 2, 3, 4], value = [2, 3, 4])
metric_selector = widgets.Dropdown(options = ["Exclusive", "Inclusive"], value = "Exclusive")
topn_selector = widgets.BoundedIntText(min = 1, max = 200, value = 25)
interact(runtime_breakdownplot_widget, trials = trial_selector, metric = metric_selector, topn = topn_selector)

In [ ]:
# FUNCTION HISTOGRAM

import re
import plotly.express as px
trial_ids = [0, 1, 2, 3, 4]
timers = []
p = re.compile("\[.*\] (.*) \[.*\]")
for id in trial_ids:
    df = TauProfileParser.parse('.tau/mpi-examples/ring-sample/{}'.format(id))
    df = df.interval_data().reset_index()
    timers += [x for x in list(df["Timer"]) if "=>" not in x]

timers = list(set(timers))
timers = [t.strip() for t in timers if 'CONTEXT' not in t and 'SAMPLE' not in t]

def function_histogram_figure(trials, metric, timer, nbins):
    dfs = []
    for id in trials:
        df = TauProfileParser.parse('.tau/mpi-examples/ring-sample/{}'.format(id))
        df = df.interval_data().reset_index()
        dfs.append(df)
    
    df = pd.concat(dfs)
    df["Timer"] = df["Timer"].apply(lambda x: x.strip())
    df = df[df["Timer"] == timer][metric].reset_index().drop(columns=["index"])
    fig = px.histogram(df, x = metric, nbins = nbins)
    fig.update_layout(
        title_text = timer,
        title_x = 0.5,
        yaxis_title_text = "# Threads",
        bargap = 0.2)
    
    return fig

def function_histogram_widget(trials, metric, timer, nbins):
    return function_histogram_figure(trials, metric, timer, nbins)

trial_selector = widgets.SelectMultiple(options = trial_ids, value = [2, 3, 4])
metric_selector = widgets.Dropdown(options = ["Exclusive", "Inclusive"], value = "Exclusive")
timer_selector = widgets.Dropdown(options = timers, value = timers[0])
nbins_selector = widgets.BoundedIntText(min = 3, max = 30, value = 5)

interact(function_histogram_widget, trials = trial_selector, metric = metric_selector, timer = timer_selector, nbins = nbins_selector)

In [ ]:
# CORRELATION PLOT

import plotly.express as px
trial_ids = [0, 1, 2, 3, 4]
metrics = ["Exclusive", "Inclusive"]

timers = []
for id in trial_ids:
    df = TauProfileParser.parse('.tau/mpi-examples/ring-sample/{}'.format(id))
    df = df.interval_data().reset_index()
    timers += [x for x in list(df["Timer"]) if "=>" not in x]

timers = list(set(timers))
timers = [t.strip() for t in timers if 'CONTEXT' not in t and 'SAMPLE' not in t]

def correlation_figure(trials, timer_1, metric_1, timer_2, metric_2):
    dfs = []
    for id in trials:
        df = TauProfileParser.parse('.tau/mpi-examples/ring-sample/{}'.format(id))
        df = df.interval_data().reset_index()
        df["Timer"] = df["Timer"].apply(lambda x: x.strip())
        df = df[df["Timer"].isin([timer_1, timer_2])]
        df["Measurement"] = [d[metric_1] if d["Timer"] == timer_1 else d[metric_2] for _, d in df.iterrows()]
        df = df[["Node", "Context", "Thread", "Timer", "Measurement"]]
        df["Tag"] = ["{} {} {}".format(d["Node"], d["Context"], d["Thread"]) for _, d in df.iterrows()]
        df = df.pivot(index="Tag", columns="Timer", values="Measurement")
        dfs.append(df)

    df = pd.concat(dfs)
    fig = px.scatter(df, x = timer_1, y = timer_2)
    
    return fig

def correlation_widget(trials, timer_1, metric_1, timer_2, metric_2):
    return correlation_figure(trials, timer_1, metric_1, timer_2, metric_2)

trial_selector = widgets.SelectMultiple(options = trial_ids, value = trial_ids)
timer_selector_1 = widgets.Dropdown(options = timers, value = timers[0])
metric_selector_1 = widgets.Dropdown(options = metrics, value = metrics[0])

timer_selector_2 = widgets.Dropdown(options = timers, value = timers[1])
metric_selector_2 = widgets.Dropdown(options = metrics, value = metrics[1])

interact(correlation_widget,
         trials = trial_selector,
         timer_1 = timer_selector_1,
         metric_1 = metric_selector_1,
         timer_2 = timer_selector_2,
         metric_2 = metric_selector_2)

In [46]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

colors_str = "#1f77b4,#aec7e8,#ff7f0e,#ffbb78,#2ca02c,#98df8a,#d62728,#ff9896,#9467bd,#c5b0d5,#8c564b,#c49c94,#e377c2,#f7b6d2,#7f7f7f,#c7c7c7,#bcbd22,#dbdb8d,#17becf,#9edae5,#7fc97f,#beaed4,#fdc086,#ffff99,#386cb0,#f0027f,#bf5b17,#666666,#1b9e77,#d95f02,#7570b3,#e7298a,#66a61e,#e6ab02,#a6761d,#666666,#fbb4ae,#b3cde3,#ccebc5,#decbe4,#fed9a6,#ffffcc,#e5d8bd,#fddaec,#f2f2f2,#393b79,#5254a3,#6b6ecf,#9c9ede,#637939,#8ca252,#b5cf6b,#cedb9c,#8c6d31,#bd9e39,#e7ba52,#e7cb94,#843c39,#ad494a,#d6616b,#e7969c,#7b4173,#a55194,#ce6dbd,#de9ed6,#8dd3c7,#ffffb3,#bebada,#fb8072,#80b1d3,#fdb462,#b3de69,#fccde5,#d9d9d9,#bc80bd,#ccebc5,#ffed6f,#3182bd,#6baed6,#9ecae1,#c6dbef,#e6550d,#fd8d3c,#fdae6b,#fdd0a2,#31a354,#74c476,#a1d99b,#c7e9c0,#756bb1,#9e9ac8,#bcbddc,#dadaeb,#636363,#969696,#bdbdbd,#d9d9d9"
colors = colors_str.split(',')

def isFunctionTimer(x):
    return "CONTEXT" not in x and "SAMPLE" not in x and "=>" not in x and ".TAU" not in x

def barplot_3d_figure(trial_id, metric):
    df = TauProfileParser.parse('.tau/mpi-examples/ring-sample/{}'.format(trial_id))
    df = df.interval_data().reset_index()
    timers = list(set([x["Timer"] for _,x in df.iterrows() if isFunctionTimer(x["Timer"])]))
    df = df[df["Timer"].isin(timers)]
    df["Tag"] = ["{}, {}, {}".format(x["Node"], x["Context"], x["Thread"]) for _, x in df.iterrows()]
    df = df[["Tag", "Timer", metric]]
    gf = df.groupby("Timer")
    
    ds = []
    
    for timer, color in zip(timers[::-1], colors[:len(timers)]):
        group = gf.get_group(timer)
        tags = group["Tag"].tolist()
        timer_coords = [timer] * len(tags)
        mval = group[metric].tolist()
        zeros = [0] * len(tags)
        ds.append(dict(
            type='scatter3d',
            mode='lines',
            x = tags + tags[::-1] + [tags[0]],
            y = timer_coords * 2 + [timer_coords[0]],
            z = mval + zeros + [mval[0]],
            name = '',
            surfaceaxis = 1,
            surfacecolor = color,
            line = dict(
                color='black',
                width=4)))
    
    layout = dict(
        height=700,
        width=700,
        title='',
        showlegend=False,
        scene=dict(
            xaxis=dict(title='indices'),
            yaxis=dict(title=''),
            zaxis=dict(title='{}'.format(metric)),
            camera=dict(
                eye=dict(x=-1.5, y=1.5, z=0.8))
        )
    )
    fig = go.Figure(data=ds, layout=layout)
    return fig

def barplot_3d_widget(trial_id, metric):
    return barplot_3d_figure(trial_id, metric)

trial_selector = widgets.Dropdown(options=[1,2,3], value=1)
metric_selector = widgets.Dropdown(options = ["Exclusive", "Inclusive"], value = "Exclusive")
interact(barplot_3d_figure, trial_id = trial_selector, metric = metric_selector)

interactive(children=(Dropdown(description='trial_id', options=(1, 2, 3), value=1), Dropdown(description='metr…

<function __main__.barplot_3d_figure(trial_id, metric)>

In [77]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pandas as pd
import numpy as np
import json

colors_str = "#1f77b4,#aec7e8,#ff7f0e,#ffbb78,#2ca02c,#98df8a,#d62728,#ff9896,#9467bd,#c5b0d5,#8c564b,#c49c94,#e377c2,#f7b6d2,#7f7f7f,#c7c7c7,#bcbd22,#dbdb8d,#17becf,#9edae5,#7fc97f,#beaed4,#fdc086,#ffff99,#386cb0,#f0027f,#bf5b17,#666666,#1b9e77,#d95f02,#7570b3,#e7298a,#66a61e,#e6ab02,#a6761d,#666666,#fbb4ae,#b3cde3,#ccebc5,#decbe4,#fed9a6,#ffffcc,#e5d8bd,#fddaec,#f2f2f2,#393b79,#5254a3,#6b6ecf,#9c9ede,#637939,#8ca252,#b5cf6b,#cedb9c,#8c6d31,#bd9e39,#e7ba52,#e7cb94,#843c39,#ad494a,#d6616b,#e7969c,#7b4173,#a55194,#ce6dbd,#de9ed6,#8dd3c7,#ffffb3,#bebada,#fb8072,#80b1d3,#fdb462,#b3de69,#fccde5,#d9d9d9,#bc80bd,#ccebc5,#ffed6f,#3182bd,#6baed6,#9ecae1,#c6dbef,#e6550d,#fd8d3c,#fdae6b,#fdd0a2,#31a354,#74c476,#a1d99b,#c7e9c0,#756bb1,#9e9ac8,#bcbddc,#dadaeb,#636363,#969696,#bdbdbd,#d9d9d9"
colors = colors_str.split(',')

def isFunctionTimer(x):
    return "CONTEXT" not in x and "SAMPLE" not in x and "=>" not in x and ".TAU" not in x

def barplot_3d_figure(trial_id, metric):
    df = TauProfileParser.parse('.tau/mpi-examples/ring-sample/{}'.format(trial_id))
    df = df.interval_data().reset_index()
    timers = sorted(list(set([x["Timer"] for _,x in df.iterrows() if isFunctionTimer(x["Timer"])])))
    timersMap = {k:i for i,k in enumerate(timers)}
    df = df[df["Timer"].isin(timers)]
    df["Tag"] = ["{}, {}, {}".format(x["Node"], x["Context"], x["Thread"]) for _, x in df.iterrows()]
    tags = sorted(list(set([x["Tag"] for _,x in df.iterrows()])))
    tagsMap = {k:i for i,k in enumerate(tags)}
    df = df[["Tag", "Timer", metric]]
    gf = df.groupby("Timer")
    
    ds = []
    
    for timer, color in zip(timers[::-1], colors[:len(timers)]):
        group = gf.get_group(timer)
        tags = group["Tag"].tolist()
        timer_coords = [timer] * len(tags)
        mval = group[metric].tolist()
        zeros = [0] * len(tags)
        xs = tags + tags[::-1] + [tags[0]]
        ys = timer_coords * 2 + [timer_coords[0]]
        dzs = mval + zeros + [mval[0]]

        ds.append({
            "label": ys[0],
            "x": [tagsMap[x] for x in xs],
            "y": [timersMap[ys[0]]]*len(xs),
            "dz": dzs,
        })
    
    fig = plt.figure(figsize=(8, 3))
    ax = fig.add_subplot(111, projection='3d')
    
    for d in ds:
        xs = d['x']
        ys = d['y']
        zs = np.zeros(len(ys))
        dxs = [0.5 for _ in range(len(ys))]
        dys = dxs
        dzs = d['dz']
        ax.bar3d(xs, ys, zs, dxs, dys, dzs, alpha=0.7)
    
    plt.xticks(ticks=range(len(tags)), labels=tags)
    plt.yticks(ticks=range(len(timers)), labels=timers, rotation=-75)
    #ax.set_yticklabels([t.strip() for t in timers], rotation=-20, va='center', ha='left')
    ax.set_zlabel(metric)
    ax.view_init(elev=56, azim=-18)
    ax.set_title("")
    plt.show()

def barplot_3d_widget(trial_id, metric):
    return barplot_3d_figure(trial_id, metric)

trial_selector = widgets.Dropdown(options=[1,2,3], value=1)
metric_selector = widgets.Dropdown(options = ["Exclusive", "Inclusive"], value = "Exclusive")
interact(barplot_3d_figure, trial_id = trial_selector, metric = metric_selector)

interactive(children=(Dropdown(description='trial_id', options=(1, 2, 3), value=1), Dropdown(description='metr…

<function __main__.barplot_3d_figure(trial_id, metric)>